## Algoritmo de automatización 



In [3]:
import json
import numpy as np
import logging
import os
import math
import copy
import random
import string
from datetime import datetime, timezone
from typing import Dict, List, Any, Optional, Tuple, Set

# --- Configuration ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# --- File Paths ---
RELAY_PAIRS_PATH = "/Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/independent_relay_pairs.json"
OPTIMIZED_SETTINGS_OUTPUT_PATH = "/Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/raw/optimized_relay_values_v9.json"

# --- Constants ---
K = 0.14
N = 0.02
CTI = 0.2
# Optimization Bounds and Parameters
MIN_TDS = 0.05
MAX_TDS = 1
MIN_PICKUP = 0.05
MAX_PICKUP_FACTOR = 0.7
MAX_TIME = 20.0
MAX_ITERATIONS = 250
# Convergence Targets (Normal Mode)
TARGET_TMT = -0.005
MIN_ALLOWED_INDIVIDUAL_MT = -0.009
# Convergence Targets (Island Mode)
TARGET_TMT_ISLAND = -0.05
MIN_ALLOWED_INDIVIDUAL_MT_ISLAND = -0.02
CONVERGENCE_THRESHOLD_TMT = 0.005
# Objective Function Weights (Normal Mode)
ALPHA_1 = 0.1  # Reemplazando W_TIME
ALPHA_2 = 15.0  # Reemplazando W_MT
BETA_2 = 0.5  # Nuevo parámetro para la función objetivo
# Objective Function Weights (Island Mode)
ALPHA_1_ISLAND = 0.5  # Reemplazando W_TIME_ISLAND
ALPHA_2_ISLAND = 5.0  # Reemplazando W_MT_ISLAND
# Adjustment Step Sizes & Factors (Normal Mode)
AGGRESSIVE_MT_THRESHOLD = -CTI * 0.75
AGGRESSIVE_TDS_BACKUP_FACTOR = 1.15
AGGRESSIVE_TDS_MAIN_FACTOR = 0.90
AGGRESSIVE_PICKUP_BACKUP_FACTOR = 1.05
NORMAL_TDS_BACKUP_ADD = 0.02
NORMAL_TDS_MAIN_SUB = 0.01
NORMAL_PICKUP_BACKUP_FACTOR = 1.01
NORMAL_PICKUP_MAIN_FACTOR = 0.99
# Adjustment Step Sizes & Factors (Island Mode - More Conservative)
AGGRESSIVE_TDS_BACKUP_FACTOR_ISLAND = 1.05
AGGRESSIVE_TDS_MAIN_FACTOR_ISLAND = 0.95
AGGRESSIVE_PICKUP_BACKUP_FACTOR_ISLAND = 1.02
NORMAL_TDS_BACKUP_ADD_ISLAND = 0.01
NORMAL_TDS_MAIN_SUB_ISLAND = 0.005
NORMAL_PICKUP_BACKUP_FACTOR_ISLAND = 1.005
NORMAL_PICKUP_MAIN_FACTOR_ISLAND = 0.995
# Island Mode Detection
ISLAND_MODE_TMT_THRESHOLD = -10.0
EXCLUDED_RELAYS = {f"R{i}" for i in [2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 
                                     37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52]}

# --- Helper Functions ---
def load_json_file(file_path: str) -> Optional[Any]:
    try:
        with open(file_path, 'r') as file: data = json.load(file)
        logger.info(f"Archivo cargado: {file_path}"); return data
    except FileNotFoundError: logger.error(f"No encontrado: {file_path}"); return None
    except json.JSONDecodeError as e: logger.error(f"JSON inválido: {file_path}: {e}"); return None
    except Exception as e: logger.error(f"Error carga {file_path}: {e}"); return None

def calculate_operation_time(I_shc: float, I_pi: float, TDS: float) -> Optional[float]:
    if I_pi <= 0 or I_shc <= 0 or TDS < MIN_TDS or TDS > MAX_TDS:
        return None
    if I_pi > I_shc * MAX_PICKUP_FACTOR:
        return MAX_TIME
    M = I_shc / I_pi
    if M <= 1.0:
        return MAX_TIME
    try:
        denominator = M**N - 1
        if abs(denominator) < 1e-9:
            return None
        time = TDS * (K / denominator)
        if not np.isfinite(time) or time <= 0:
            return None
        return min(time, MAX_TIME)
    except (OverflowError, ValueError):
        return None
    except Exception as e:
        logger.error(f"Excepción inesperada en calc_op_time (I_shc={I_shc}, I_pi={I_pi}, TDS={TDS}): {e}")
        return None

def group_data_by_scenario(relay_pairs_data: List[Dict]) -> Dict[str, Dict[str, Any]]:
    scenario_map: Dict[str, Dict[str, Any]] = {}
    processed_pairs_count = 0
    skipped_pairs_count = 0
    island_scenarios = set()
    
    for pair_entry in relay_pairs_data:
        scenario_id = pair_entry.get("scenario_id")
        tmt = pair_entry.get("TMT")
        if scenario_id and isinstance(tmt, (int, float)) and tmt < ISLAND_MODE_TMT_THRESHOLD:
            island_scenarios.add(scenario_id)
            logger.info(f"Escenario {scenario_id} identificado como modo isla (TMT={tmt:.4f} < {ISLAND_MODE_TMT_THRESHOLD})")

    for i, pair_entry in enumerate(relay_pairs_data):
        scenario_id = pair_entry.get("scenario_id")
        main_relay_info = pair_entry.get('main_relay')
        backup_relay_info = pair_entry.get('backup_relay')
        if not scenario_id or not isinstance(main_relay_info, dict) or not isinstance(backup_relay_info, dict):
            skipped_pairs_count += 1
            continue
        if scenario_id not in scenario_map:
            scenario_map[scenario_id] = {"pairs_info": [], "initial_settings": {}, "relays": set(), "is_island_mode": scenario_id in island_scenarios}
        main_relay = main_relay_info.get('relay')
        backup_relay = backup_relay_info.get('relay')
        I_shc_main = main_relay_info.get('Ishc')
        I_shc_backup = backup_relay_info.get('Ishc')

        if scenario_id in island_scenarios and (main_relay in EXCLUDED_RELAYS or backup_relay in EXCLUDED_RELAYS):
            logger.debug(f"Omitiendo par {main_relay}/{backup_relay} en {scenario_id} (modo isla, relé excluido)")
            skipped_pairs_count += 1
            continue

        if not (main_relay and backup_relay and
                isinstance(I_shc_main, (int, float)) and I_shc_main > 0 and
                isinstance(I_shc_backup, (int, float)) and I_shc_backup > 0):
            skipped_pairs_count += 1
            continue

        processed_pairs_count += 1
        scenario_map[scenario_id]['relays'].add(main_relay)
        scenario_map[scenario_id]['relays'].add(backup_relay)
        initial_settings_scenario = scenario_map[scenario_id]['initial_settings']
        for r_name, r_info in [(main_relay, main_relay_info), (backup_relay, backup_relay_info)]:
            if r_name not in initial_settings_scenario:
                tds = r_info.get('TDS')
                pickup = r_info.get('pick_up')
                if isinstance(tds, (int, float)) and isinstance(pickup, (int, float)):
                    initial_settings_scenario[r_name] = {
                        'TDS_initial': float(tds),
                        'pickup_initial': float(pickup)
                    }
        scenario_map[scenario_id]['pairs_info'].append({
            "main_relay": main_relay,
            "backup_relay": backup_relay,
            "I_shc_main": float(I_shc_main),
            "I_shc_backup": float(I_shc_backup)
        })

    logger.info(f"Datos agrupados por escenario. Pares procesados: {processed_pairs_count}, Pares omitidos: {skipped_pairs_count}")
    return scenario_map

def run_scenario_optimization(
    scenario_id: str,
    pairs_info: List[Dict],
    initial_settings: Dict[str, Dict[str, float]],
    relays_in_scenario: Set[str],
    is_island_mode: bool
) -> Dict[str, Dict[str, float]]:
    logger.info(f"--- Iniciando optimización para {scenario_id} ---")
    if not pairs_info:
        logger.warning(f"({scenario_id}) No hay pares válidos.")
        return {}

    relay_settings = {}
    default_pickup = MIN_PICKUP * 1.5
    for relay in relays_in_scenario:
        if relay in initial_settings:
            relay_settings[relay] = {
                "TDS": MIN_TDS,
                "pickup": max(MIN_PICKUP, initial_settings[relay]['pickup_initial'])
            }
        else:
            logger.warning(f"({scenario_id}) Config inicial (pickup) no encontrada para '{relay}'. Usando defecto.")
            relay_settings[relay] = {"TDS": MIN_TDS, "pickup": default_pickup}

    last_tmt = float('inf')
    no_improvement_streak = 0

    # Select optimization parameters based on mode
    alpha_1 = ALPHA_1_ISLAND if is_island_mode else ALPHA_1
    alpha_2 = ALPHA_2_ISLAND if is_island_mode else ALPHA_2
    target_tmt = TARGET_TMT_ISLAND if is_island_mode else TARGET_TMT
    min_allowed_individual_mt = MIN_ALLOWED_INDIVIDUAL_MT_ISLAND if is_island_mode else MIN_ALLOWED_INDIVIDUAL_MT
    aggressive_tds_backup_factor = AGGRESSIVE_TDS_BACKUP_FACTOR_ISLAND if is_island_mode else AGGRESSIVE_TDS_BACKUP_FACTOR
    aggressive_tds_main_factor = AGGRESSIVE_TDS_MAIN_FACTOR_ISLAND if is_island_mode else AGGRESSIVE_TDS_MAIN_FACTOR
    aggressive_pickup_backup_factor = AGGRESSIVE_PICKUP_BACKUP_FACTOR_ISLAND if is_island_mode else AGGRESSIVE_PICKUP_BACKUP_FACTOR
    normal_tds_backup_add = NORMAL_TDS_BACKUP_ADD_ISLAND if is_island_mode else NORMAL_TDS_BACKUP_ADD
    normal_tds_main_sub = NORMAL_TDS_MAIN_SUB_ISLAND if is_island_mode else NORMAL_TDS_MAIN_SUB
    normal_pickup_backup_factor = NORMAL_PICKUP_BACKUP_FACTOR_ISLAND if is_island_mode else NORMAL_PICKUP_BACKUP_FACTOR
    normal_pickup_main_factor = NORMAL_PICKUP_MAIN_FACTOR_ISLAND if is_island_mode else NORMAL_PICKUP_MAIN_FACTOR

    logger.info(f"({scenario_id}) Usando parámetros {'de modo isla' if is_island_mode else 'normales'}: "
                f"ALPHA_1={alpha_1}, ALPHA_2={alpha_2}, TARGET_TMT={target_tmt}, MIN_ALLOWED_INDIVIDUAL_MT={min_allowed_individual_mt}")

    for iteration in range(MAX_ITERATIONS):
        tmt = 0.0
        error_pairs_count = 0
        max_neg_mt = 0
        current_pair_results = []

        # Primera parte de la función objetivo: suma de tiempos de operación al cuadrado
        operation_times_sum = 0.0
        relays_processed = set()  # Para evitar duplicar relés

        # Segunda parte: suma de términos de descoordinación al cuadrado
        miscoordination_sum = 0.0

        for pair in pairs_info:
            main_relay = pair["main_relay"]
            backup_relay = pair["backup_relay"]
            I_shc_main = pair["I_shc_main"]
            I_shc_backup = pair["I_shc_backup"]
            if main_relay not in relay_settings or backup_relay not in relay_settings:
                continue

            tds_main = relay_settings[main_relay]["TDS"]
            pickup_main = relay_settings[main_relay]["pickup"]
            tds_backup = relay_settings[backup_relay]["TDS"]
            pickup_backup = relay_settings[backup_relay]["pickup"]

            pickup_main_bounded = max(MIN_PICKUP, pickup_main)
            pickup_backup_bounded = max(MIN_PICKUP, pickup_backup)

            main_time = calculate_operation_time(I_shc_main, pickup_main_bounded, tds_main)
            backup_time = calculate_operation_time(I_shc_backup, pickup_backup_bounded, tds_backup)

            # Acumular tiempos de operación para la primera parte de OF (solo una vez por relé)
            if main_time is not None and main_time < MAX_TIME and main_relay not in relays_processed:
                operation_times_sum += main_time ** 2
                relays_processed.add(main_relay)
            if backup_time is not None and backup_time < MAX_TIME and backup_relay not in relays_processed:
                operation_times_sum += backup_time ** 2
                relays_processed.add(backup_relay)

            delta_t: Optional[float] = None
            mt: Optional[float] = None

            if main_time is None or backup_time is None:
                delta_t = None
                mt = None
                error_pairs_count += 1
            elif main_time >= MAX_TIME or backup_time >= MAX_TIME:
                delta_t = backup_time - main_time if main_time < MAX_TIME and backup_time < MAX_TIME else MAX_TIME
                mt = MAX_TIME * 2
            else:
                delta_t = backup_time - main_time
                mt = delta_t - CTI

            current_pair_results.append({
                "main_time": main_time,
                "backup_time": backup_time,
                "delta_t": delta_t,
                "mt": mt,
                "main_relay": main_relay,
                "backup_relay": backup_relay,
                "I_shc_main": I_shc_main,
                "I_shc_backup": I_shc_backup
            })

            if mt is not None and mt < 0 and mt < MAX_TIME:
                tmt += mt
                # Calcular el término de descoordinación: Δt_mbj - β_2 * (Δt_mbj - |Δt_mbj|)
                # Si Δt_mbj < 0, entonces |Δt_mbj| = -Δt_mbj, por lo que Δt_mbj - |Δt_mbj| = Δt_mbj - (-Δt_mbj) = 2Δt_mbj
                # Si Δt_mbj >= 0, entonces Δt_mbj - |Δt_mbj| = Δt_mbj - Δt_mbj = 0
                delta_t_val = delta_t
                if delta_t_val < 0:
                    miscoordination_term = delta_t_val - BETA_2 * (delta_t_val - abs(delta_t_val))
                else:
                    miscoordination_term = delta_t_val
                miscoordination_sum += miscoordination_term ** 2

                max_neg_mt = min(max_neg_mt, mt)

        # Calcular la función objetivo OF
        of = alpha_1 * operation_times_sum + alpha_2 * miscoordination_sum

        if (iteration + 1) % 25 == 0 or iteration == 0 or tmt > target_tmt*1.1:
            logger.info(f"({scenario_id}) Iter {iteration+1}/{MAX_ITERATIONS}: OF={of:.4f}, TMT={tmt:.4f}, MaxNegMT={max_neg_mt:.4f}, Errors={error_pairs_count}")

        if tmt >= target_tmt and max_neg_mt >= min_allowed_individual_mt:
            logger.info(f"({scenario_id}) Convergencia alcanzada (TMT={tmt:.4f} >= {target_tmt}, MaxNegMT={max_neg_mt:.4f} >= {min_allowed_individual_mt}).")
            break

        adjustments_made = False
        next_relay_settings = copy.deepcopy(relay_settings)

        for pair_res in current_pair_results:
            if pair_res["mt"] is None or pair_res["mt"] >= MAX_TIME:
                continue
            if pair_res["mt"] < 0:
                main_relay = pair_res["main_relay"]
                backup_relay = pair_res["backup_relay"]
                I_shc_main = pair_res["I_shc_main"]
                I_shc_backup = pair_res["I_shc_backup"]
                mt_val = pair_res["mt"]

                tds_main_curr = next_relay_settings[main_relay]["TDS"]
                tds_backup_curr = next_relay_settings[backup_relay]["TDS"]
                pickup_main_curr = next_relay_settings[main_relay]["pickup"]
                pickup_backup_curr = next_relay_settings[backup_relay]["pickup"]

                new_tds_backup = tds_backup_curr
                new_pickup_backup = pickup_backup_curr
                new_tds_main = tds_main_curr
                new_pickup_main = pickup_main_curr

                if mt_val < AGGRESSIVE_MT_THRESHOLD:
                    new_tds_backup = min(MAX_TDS, tds_backup_curr * aggressive_tds_backup_factor)
                    if abs(new_tds_backup - MAX_TDS) < 1e-6:
                        max_allowed_pickup_backup = I_shc_backup * MAX_PICKUP_FACTOR
                        new_pickup_backup = min(max_allowed_pickup_backup, max(MIN_PICKUP, pickup_backup_curr * aggressive_pickup_backup_factor))
                    new_tds_main = max(MIN_TDS, tds_main_curr * aggressive_tds_main_factor)
                else:
                    new_tds_backup = min(MAX_TDS, max(MIN_TDS, tds_backup_curr + normal_tds_backup_add))
                    new_tds_main = min(MAX_TDS, max(MIN_TDS, tds_main_curr - normal_tds_main_sub))

                next_relay_settings[backup_relay]["TDS"] = min(MAX_TDS, max(MIN_TDS, new_tds_backup))
                next_relay_settings[main_relay]["TDS"] = min(MAX_TDS, max(MIN_TDS, new_tds_main))
                max_pickup_b = I_shc_backup * MAX_PICKUP_FACTOR
                max_pickup_m = I_shc_main * MAX_PICKUP_FACTOR
                next_relay_settings[backup_relay]["pickup"] = min(max_pickup_b, max(MIN_PICKUP, new_pickup_backup))
                next_relay_settings[main_relay]["pickup"] = min(max_pickup_m, max(MIN_PICKUP, new_pickup_main))

                adjustments_made = True

        if adjustments_made:
            relay_settings = next_relay_settings
        elif iteration > 15:
            logger.info(f"({scenario_id}) No se realizaron ajustes en iteración {iteration + 1}. Deteniendo.")
            break

        if abs(tmt - last_tmt) < CONVERGENCE_THRESHOLD_TMT:
            no_improvement_streak += 1
        else:
            no_improvement_streak = 0
        last_tmt = tmt

        if no_improvement_streak >= 25:
            logger.warning(f"({scenario_id}) TMT no mejora significativamente ({no_improvement_streak} iteraciones). Deteniendo.")
            break
    else:
        logger.warning(f"({scenario_id}) La optimización no convergió después de {MAX_ITERATIONS} iteraciones.")
        logger.warning(f"({scenario_id}) Estado final: TMT={tmt:.4f}, MaxNegMT={max_neg_mt:.4f}, Errors={error_pairs_count}")

    formatted_settings = {}
    relays_max_ishc = {}
    for relay in relays_in_scenario:
        max_ishc = 0
        for p in pairs_info:
            if p['main_relay'] == relay:
                max_ishc = max(max_ishc, p.get('I_shc_main', 0))
            if p['backup_relay'] == relay:
                max_ishc = max(max_ishc, p.get('I_shc_backup', 0))
        relays_max_ishc[relay] = max_ishc if max_ishc > 0 else (default_pickup / MIN_PICKUP)

    for relay, settings in relay_settings.items():
        final_pickup = settings['pickup']
        max_ishc_relay = relays_max_ishc.get(relay, default_pickup / MIN_PICKUP)
        final_pickup_bounded = min(max_ishc_relay * MAX_PICKUP_FACTOR, max(MIN_PICKUP, final_pickup))
        final_tds_bounded = min(MAX_TDS, max(MIN_TDS, settings['TDS']))
        formatted_settings[relay] = {
            "TDS": float(f"{final_tds_bounded:.5f}"),
            "pickup": float(f"{final_pickup_bounded:.5f}")
        }
    logger.info(f"--- Optimización finalizada para {scenario_id} ---")
    return formatted_settings

if __name__ == "__main__":
    logger.info("--- Iniciando Script de Optimización de Ajustes de Relés (v8 - Adjusted Island Mode Optimization) ---")
    if not os.path.exists(RELAY_PAIRS_PATH):
        logger.error(f"¡Error Crítico! No se encontró el archivo de entrada: {RELAY_PAIRS_PATH}")
        raise SystemExit("Archivo de pares de relés no encontrado.")

    relay_pairs_data = load_json_file(RELAY_PAIRS_PATH)
    if relay_pairs_data is None:
        raise SystemExit("Error crítico al cargar archivo de pares.")

    scenario_data_map = group_data_by_scenario(relay_pairs_data)
    if not scenario_data_map:
        raise SystemExit("Error crítico al procesar datos de entrada. No se encontraron escenarios o pares válidos.")

    all_optimized_settings = {}
    successful_scenarios = 0
    failed_scenarios = 0
    for scenario_id, scenario_data in scenario_data_map.items():
        if not scenario_data.get('relays') or not scenario_data.get('pairs_info'):
            logger.warning(f"Omitiendo escenario '{scenario_id}': No contiene relés o pares de información válidos después del procesamiento inicial.")
            failed_scenarios += 1
            continue
        try:
            optimized_settings_for_scenario = run_scenario_optimization(
                scenario_id,
                scenario_data['pairs_info'],
                scenario_data['initial_settings'],
                scenario_data['relays'],
                scenario_data['is_island_mode']
            )
            if optimized_settings_for_scenario:
                all_optimized_settings[scenario_id] = optimized_settings_for_scenario
                successful_scenarios += 1
            else:
                logger.warning(f"La optimización no produjo resultados para el escenario: {scenario_id}")
                failed_scenarios += 1
        except Exception as e:
            logger.error(f"Error inesperado durante optimización del escenario {scenario_id}: {e}", exc_info=True)
            failed_scenarios += 1

    output_list = []
    if all_optimized_settings:
        logger.info(f"Optimización completada. Escenarios exitosos: {successful_scenarios}, Escenarios fallidos/omitidos: {failed_scenarios}.")
        logger.info("Formateando resultados optimizados en la estructura de lista deseada...")
        for scenario_id, optimized_settings in all_optimized_settings.items():
            current_timestamp = datetime.now(timezone.utc).isoformat(timespec='microseconds').replace('+00:00', 'Z')
            list_entry = {
                "scenario_id": scenario_id,
                "timestamp": current_timestamp,
                "relay_values": optimized_settings
            }
            output_list.append(list_entry)
        logger.info(f"Formato de lista creado con {len(output_list)} escenarios optimizados.")

        try:
            output_dir = os.path.dirname(OPTIMIZED_SETTINGS_OUTPUT_PATH)
            if output_dir:
                os.makedirs(output_dir, exist_ok=True)
            with open(OPTIMIZED_SETTINGS_OUTPUT_PATH, 'w') as file:
                json.dump(output_list, file, indent=2, allow_nan=False)
            logger.info(f"Archivo con ajustes optimizados (v8, adjusted island mode optimization) guardado en: {OPTIMIZED_SETTINGS_OUTPUT_PATH}")
        except Exception as e:
            logger.error(f"Error al guardar el archivo de salida {OPTIMIZED_SETTINGS_OUTPUT_PATH}: {e}")
    else:
        logger.error(f"La optimización falló o no produjo resultados para ningún escenario ({successful_scenarios} exitosos, {failed_scenarios} fallidos/omitidos). No se guardó ningún archivo.")

    logger.info("--- Script de Optimización Finalizado ---")

2025-04-20 14:13:00,945 - INFO - --- Iniciando Script de Optimización de Ajustes de Relés (v8 - Adjusted Island Mode Optimization) ---
2025-04-20 14:13:00,989 - INFO - Archivo cargado: /Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/independent_relay_pairs.json
2025-04-20 14:13:01,027 - INFO - Datos agrupados por escenario. Pares procesados: 6720, Pares omitidos: 80
2025-04-20 14:13:01,037 - INFO - --- Iniciando optimización para scenario_1 ---
2025-04-20 14:13:01,038 - INFO - (scenario_1) Usando parámetros normales: ALPHA_1=0.1, ALPHA_2=15.0, TARGET_TMT=-0.005, MIN_ALLOWED_INDIVIDUAL_MT=-0.009
2025-04-20 14:13:01,040 - INFO - (scenario_1) Iter 1/250: OF=1.8954, TMT=-23.8788, MaxNegMT=-0.6159, Errors=0
2025-04-20 14:13:01,052 - INFO - (scenario_1) Iter 25/250: OF=13.8604, TMT=-6.2130, MaxNegMT=-0.4648, Errors=0
2025-04-20 14:13:01,062 - INFO - (scenario_1) Iter 50/250: OF=16.8900, TMT=-3.2420, MaxNegMT=-0.2350, Errors=0
2025-04-20 14:13:01,072 - INFO - (s